# Multiprocessing: limited direct execution; system calls
_COSC 208, Introduction to Computer Systems, 2024-04-05_

## Announcements
* Project 2 revisions due Wednesday at 11pm
* Quiz 5 rescheduled for Friday, April 19
* Office hours
    * Monday – canceled
    * Tuesday – 8:15am-10am and 12:45pm-1:15pm

## Outline
* Operating Systems (OS)
* Accessing hardware
* Limited Direct Execution (LDE)
* System calls (syscalls)

## No warm-up: Happy Friday!

<div style="page-break-after:always;"></div>

## Operating systems (OS) overview

* Purpose of an OS
    * Make computer hardware easy to use—e.g., an OS knows how to load an application's executable code from persistent storage (e.g., solid state drive (SSD)) into main memory, initialize the process's address space (code, heap, stack), and make the CPU execute the application's instructions
    * Support multiprocessing—i.e., running multiple applications concurrently
        * Concurrently == switch between multiple tasks during a window of time—e.g., alternate between cooking and setting the table
        * Simultaneously == complete two tasks at the same time—e.g., listen to a podcast while cooking
    * Allocate and manage hardware resources—e.g., decide when/which applications can use the CPU, decide when/which memory applications can use, prevent applications from stealing/accessing another application's CPU time or memory
    * Many OSes also provide a user interface (UI)
* How does the OS fulfill its duties?
    * Mechanisms — fundamental approaches for managing/providing access to hardware resources
        * E.g., system calls, process abstraction
    * Policy — specific ways of employing an approach; different policies make different trade-offs (in terms of efficiency, performance, etc.)
        * E.g., process scheduler

## Accessing hardware

* OS is responsible for allocating/managing the hardware
* Analogy – sharing a calculator
    * Three students – OS, program 1, program 2
    * OS decides how programs will share the calculator

* First attempt – no preemption
    * Instructions for Program 1:
        ```
        8642 + 9753 = _____
        M+ (add to memory)
        1819 + 1920 + 2021 = _____
        153 + 264 + MRC (recall from memory) = _____
        ```
    * Instructions for Program 2:
        ```
        1234 + 5678 = _____
        1470 + 2342 + 2341 = _____
        34238 + 32482 = _____
        ```

* Second attempt – preemption, but no protection
    * Instructions for Program 1:
        ```
        8642 + 9753 = _____
        M+ (add to memory)
        1819 + 1920 + 2021 = _____
        153 + 264 + MRC (recall from memory) = _____
        ```
    * Instructions for Program 2:
        ```
        8395 + 8395 = _____
        M- (subtract from memory)
        1470 + 2342 + 2341 = _____
        34238 + 32482 = _____
        ```

* Third attempt – dictate instructions
    * Instructions for Program 1:
        ```
        8642 + 9753 = _____
        1 + 2 + 3 + 4 + 5 + 6 + ... + 100 = _____
        ```
    * Instructions for Program 2:
        ```
        8395 + 8395 = _____
        1470 + 2342 + 2341 = _____
        34238 + 32482 = _____
        ```

* OS is responsible for allocating/managing the hardware
    * ⇒ applications should **not have unfettered access to hardware**
* _How should applications access the hardware?_
    * Ask the OS for access to the hardware
        * How do we ensure the OS does not "lose control" of the hardware? 
    * Asks the OS to perform an action on the application's behalf
        * How do we ensure this doesn't substantially degrade performance?
* Example: execute an instruction on the CPU
    * Asking the OS to do this on behalf of an application is impractical—OS would need to execute multiple assembly instructions for each assembly instruction the application wants to execute
    * Alternative: allow the application to execute its own instructions on the CPU
        * This is risky—an application may execute an instruction that controls the hardware, e.g., `hlt` (halt) instruction pauses the CPU
    * Alternative: allow the application to execute "safe" instructions on its own on the CPU
* Example: accessing the solid state drive (SSD)
    * Allowing the application to access the SSD directly
        * This is risky—an application may read/write data it should not be able to access
    * Alternative: asking the OS to access the SSD on the application's behalf
        * Latency of accessing SSD (~1 million CPU cycles) far outweighs the extra instructions required for OS to perform the access on the application's behalf
* Mechanisms
    * Limited Direct Execution (LDE)
    * System calls (syscalls)

## Limited Direct Execution (LDE)

* CPU has two modes of execution: user mode & kernel mode
* _When does a CPU run in user mode?_ — when executing application code
* _When does a CPU run in kernel mode?_ — when executing OS code
* Allowable operations in user mode are restricted
    * Applications can...
        * Perform arithmetic/logic operations
        * Load/store values in its stack/heap
    * Applications must ask the OS to...
        * Start/terminate applications
        * Create/delete files
        * Display output on screen
        * Read input from user
    * Must transfer control to the OS to perform these operations — How?

## System calls (syscalls)

* Invoked via a special assembly instruction: trap (generic) or `svc` (on ARM)

* Example program

In [1]:
#include <stdio.h>
#include <unistd.h>
int name() {
    int pid = getpid();
    return pid;
}
int main() {
    int p = name();
    printf("I am a process. My name is %d.\n", p);
}

I am a process. My name is 1276.


* Assembly code

```
00000000004006ac <name>:
    4006ac:    d10083ff     sub    sp, sp, #0x20
    4006b0:    f9000bfe     str    x30, [sp, #16]
    4006b4:    94007713     bl    41e300 <__getpid>
    4006b8:    b9000fe0     str    w0, [sp, #12]
    4006bc:    b9400fe0     ldr    w0, [sp, #12]
    4006c0:    f9400bfe     ldr    x30, [sp, #16]
    4006c4:    910083ff     add    sp, sp, #0x20
    4006c8:    d65f03c0     ret
000000000041e300 <__getpid>:
    41e300:    d28015c8     mov    x8, #0xac
    41e304:    d4000001     svc    #0x0
    41e308:    d65f03c0     ret
```

* Functions in the C standard library that involve a privileged operation (e.g., `printf`) put the system call number in a register and invoke a trap instruction — programmer doesn’t have to worry about these details; they can just call the appropriate function in the C standard library
* When `svc` is executed
    1. CPU switches to kernel mode
    2. CPU saves registers to the kernel stack — kernel stack is at a fixed location in memory
        * _Why do we need to save the registers?_ — so we can return to `user` when `__getuid` is done
    3. CPU uses system call number to index into table of trap handlers 
        * At boot, initialize table of trap handlers with pointers into OS code for handling each type of syscall
    4. Branch and link to trap handler code
    5. CPU restores registers from the kernel stack
    6. CPU switches to user mode
    7. Resume execution after `svc` 
* _What should we do if an application tries to perform a privileged operation without making a system call?_
    * Let the code keep running — code may assume privileged operation was successful
    * Terminate the application